In [1]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    project_id="2720f785-a3a2-4472-969d-19a53891c817"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

In [2]:
from entitysdk.models import Circuit
client.get_entity(entity_id="662dce5c-306d-4737-acd6-a3af985aad7f", entity_type=Circuit)

Circuit(assets=[Asset(id=UUID('87df605e-0bd7-401a-a917-f5eb2380ff49'), update_date=None, creation_date=None, path='circuit', full_path='public/1f91f30e-1489-4e2a-8eb7-1217257c8e19/7a411785-6895-4839-aaa2-d9f76e09875a/assets/circuit/662dce5c-306d-4737-acd6-a3af985aad7f/circuit', is_directory=True, content_type='application/vnd.directory', size=-1, sha256_digest=None, status='created', meta={}, label=None)], id=UUID('662dce5c-306d-4737-acd6-a3af985aad7f'), update_date=datetime.datetime(2025, 6, 23, 12, 35, 7, 820955, tzinfo=TzInfo(UTC)), creation_date=datetime.datetime(2025, 6, 23, 12, 35, 7, 820955, tzinfo=TzInfo(UTC)), name='nbS1-O1-vSub-nCN-HEX0-L1-01', description='A volumetric subcircuit (vSub) extracted from the nbS1-O1 circuit, around a neuron with high centrality in the network’s connectivity graph, located in layer 1 of subcolumn HEX0. The subcircuit consists of the central neuron and its nine nearest neighbors, capturing the local connectivity within the parent circuit. The suf

# Create SimulationForm using Cirucit

In [3]:
import obi_one as obi

# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(name="O1 Simulation", description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timestamps.ref, stim_duration=800, frequency=20, neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='VoltageRecording')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str="662dce5c-306d-4737-acd6-a3af985aad7f"), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

/Users/james/Documents/obi/code/obi-one/obi_one/scientific/basic_connectivity_plots/helpers.py:20: UserWarning: Connectome functionalities not available
  warnings.warn("Connectome functionalities not available", UserWarning, stacklevel=1)
/Users/james/Documents/obi/code/obi-one/obi_one/scientific/basic_connectivity_plots/basic_connectivity_plots.py:31: UserWarning: Connectome functionalities not available
  warnings.warn("Connectome functionalities not available", UserWarning, stacklevel=1)


# Generate GridScan

In [4]:
import tempfile
import os

# Create a temporary directory
with tempfile.TemporaryDirectory() as temp_dir:
    print(f"Temporary directory created at: {temp_dir}")
    grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    # grid_scan.coordinate_instances(display=True)
    grid_scan.execute(processing_method='generate', data_postprocessing_method='save', db_client=client)

Temporary directory created at: /var/folders/pp/r2gyck8j4n9dfb_x9vjslq740000gn/T/tmpwdadxy77
initialize.circuit is a CircuitFromID instance.


EntitySDKError: HTTP error 403 for POST https://staging.openbraininstitute.org/api/entitycore/simulation/13005206-8e16-441b-a0c2-3ada96b24f2a/assets
json       : null
params     : {'label': 'simulation_generation_config'}
response   : <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>
